## Evaluate Human and Model Answers with ChatGPT - Vicuna Style

In [2]:
from datasets import load_dataset
import pandas as pd
pd.set_option('display.max_colwidth', 0)

## Create Open Assistant Evaluation Dataset

We select `OpenAssistant/oasst1` (`th`, `ja` and `vn`) as it contains natural instructions in languages our collaborators know with enough examples.

In [3]:
ds = load_dataset('OpenAssistant/oasst1')
ds

Using custom data configuration OpenAssistant--oasst1-fe5c583d2314bdd6
Found cached dataset parquet (/home/charipol/.cache/huggingface/datasets/OpenAssistant___parquet/OpenAssistant--oasst1-fe5c583d2314bdd6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'],
        num_rows: 4401
    })
    train: Dataset({
        features: ['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'],
        num_rows: 84437
    })
})

In [4]:
train_df = pd.DataFrame(ds['train'])
train_df['split'] = 'train'
valid_df = pd.DataFrame(ds['validation'])
valid_df['split'] = 'valid'
all_df = pd.concat([train_df,valid_df],axis=0).reset_index(drop=True)
all_df.shape

(88838, 19)

In [5]:
all_df.groupby('lang').message_tree_id.nunique().reset_index()\
    .sort_values('message_tree_id', ascending=False)

,lang,message_tree_id
7,es,3915
6,en,3765
18,ru,766
24,zh,386
5,de,351
10,fr,263
2,ca,250
20,th,167
17,pt-BR,165
13,it,115


In [6]:
all_df['rnk'] = all_df.sort_values(['created_date'], ascending=[True]) \
             .groupby(['message_tree_id']) \
             .cumcount()

def get_nb(d, label):
    if d is None: return 0
    if (label in d['name']):
        idx = d['name'].index(label)
    else:
        return 0
    return d['count'][idx]
    
all_df['nb_plus'] = all_df.emojis.map(lambda d: get_nb(d, '+1'))
all_df['nb_minus'] = all_df.emojis.map(lambda d: get_nb(d, '-1'))
all_df['nb_net_plus'] = all_df.nb_plus - all_df.nb_minus

In [7]:
prompt_df = all_df[(all_df.rnk==0)][['lang',
                                     'message_tree_id',
                                     'message_id',
                                     'text',
                                     'nb_net_plus']]
prompt_df = prompt_df[prompt_df.nb_net_plus>=0]
prompt_df.shape, prompt_df.lang.value_counts()

((9714, 5),
 es       3667
 en       3449
 ru       720 
 zh       339 
 de       326 
 fr       241 
 ca       239 
 th       166 
 pt-BR    152 
 it       103 
 uk-UA    73  
 eu       65  
 ja       42  
 pl       38  
 vi       37  
 hu       17  
 ar       14  
 da       11  
 ko       5   
 tr       5   
 id       2   
 fi       2   
 cs       1   
 Name: lang, dtype: int64)

In [8]:
answer_df = all_df[(all_df.rnk>0)][['lang',
                                     'message_tree_id',
                                     'parent_id',
                                     'message_id',
                                     'text',
                                     'nb_net_plus']]
answer_df = answer_df[answer_df.nb_net_plus>=0]
answer_df['answer_rnk'] = answer_df.sort_values(['nb_net_plus'], ascending=[False]) \
             .groupby(['parent_id']) \
             .cumcount()
answer_df = answer_df[answer_df.answer_rnk==0]
answer_df.parent_id.value_counts(), answer_df.shape

(c8e83833-ecbc-44fe-b6db-735228c25a1c    1
 bc3c7784-1f2b-46f2-85e6-d27f3c264be2    1
 4782cd35-b062-4040-bf54-3a2f044e03e8    1
 b6f00415-1469-46a3-b425-be49f1533787    1
 6ecd3b03-7b02-46e4-aa6c-1aaf31ca03e7    1
                                        ..
 a436b070-34e6-405e-8c55-b066c5b12e98    1
 975f47da-300a-4f45-87b7-492a89cd299e    1
 ccb34f9c-6ac9-404e-9b33-9ac594c89be3    1
 ca517fbb-d007-40c6-85f4-5c354c51581b    1
 88ad2577-484c-4e87-b4d2-e12665881012    1
 Name: parent_id, Length: 40307, dtype: int64,
 (40307, 7))

In [9]:
qa_df  = prompt_df.merge(answer_df[['parent_id','text','nb_net_plus']], 
                         left_on='message_id', right_on='parent_id').reset_index(drop=True)
qa_df.columns = ['lang','message_tree_id','message_id',
                 'prompt','prompt_nb_net_plus','parent_id',
                 'answer','answer_nb_net_plus']
qa_df = qa_df[['lang','message_tree_id','message_id','parent_id',
               'prompt_nb_net_plus','answer_nb_net_plus','prompt','answer']]
qa_df.lang.value_counts()

es       3656
en       3425
ru       708 
zh       339 
de       324 
ca       239 
fr       238 
th       166 
pt-BR    152 
it       103 
uk-UA    73  
eu       65  
ja       42  
pl       37  
vi       37  
hu       17  
ar       14  
da       11  
ko       5   
tr       5   
id       2   
fi       2   
cs       1   
Name: lang, dtype: int64

In [10]:
#choose only sizeable languages of interest
lang_of_interest = ['th','ja','vi']
selected_df = qa_df[qa_df.lang.map(lambda x: x in lang_of_interest)]\
                    .sort_values('lang').reset_index(drop=True)
selected_df.lang.value_counts()

th    166
ja    42 
vi    37 
Name: lang, dtype: int64

In [11]:
selected_df[selected_df.lang=='th'].head(20)

,lang,message_tree_id,message_id,parent_id,prompt_nb_net_plus,answer_nb_net_plus,prompt,answer
42,th,8a512668-adeb-4533-8ba3-eb93aa802e09,8a512668-adeb-4533-8ba3-eb93aa802e09,8a512668-adeb-4533-8ba3-eb93aa802e09,2,4,ลืมรหัสผ่าน Discord ของตัวเอง ทำยังไงดี,ในหน้า Login คุณกรอก email หรือ เบอร์โทร แล้วลองกด Forget your Password? \nDiscord จะส่ง Email ไปหาคุณถ้าคุณพิมพ์ Email หรือ OTP ถ้าคุณกรอกเบอร์โทร ให้ทำตามขั้นตอนที่ Discord แจ้งคุณมาต่อไป\nหากคุณเกิดปัญหาอะไรขึ้นระหว่างทำตามขั้นตอนนั้นคุณสามารถกลับมาถามฉันให่ได้
43,th,3230604e-6d99-4990-b238-4e35d8369ab6,3230604e-6d99-4990-b238-4e35d8369ab6,3230604e-6d99-4990-b238-4e35d8369ab6,1,2,ร้านที่ขายข้าวหมูเกาหลี ยากิโซบะที่ศูนย์อาหารเดอะมอลล์โคราชย้ายไปที่ไหน หรือปิดไปแล้วครับ,ฉันขอโทษด้วย เนื่องจากฉันเป็นโมเดลภาษา ฉันไม่มีข้อมูลเกี่ยวกับสถานที่ต่าง ๆ ที่ปิดตัวหรือย้ายไปแล้วที่มีความถูกต้องและเป็นปัจจุบัน\n\nหากคุณต้องการข้อมูล คุณสามารถติดต่อศูนย์การค้า ค้นหาข้อมูลสถานที่เพิ่มเติมได้จากอินเทอร์เน็ต หรือสอบถามกลุ่มพูดคุยในอินเทอร์เน็ตที่เกี่ยวกับร้านอาหารในพื้นที่ที่คุณอยู่
44,th,5596a395-c2a8-4e19-ada8-b7653aa1161b,5596a395-c2a8-4e19-ada8-b7653aa1161b,5596a395-c2a8-4e19-ada8-b7653aa1161b,1,3,คิดว่ามีโอกาสที่จะเกิดสงครามระหว่างสหรัฐกับจีนในอีก 2-5 ปีข้างหน้ามั้ย,การทำนายเกี่ยวกับการเกิดสงครามระหว่างสหรัฐกับจีนในอนาคตไม่สามารถทำได้อย่างแม่นยำ โดยเฉพาะอย่างยิ่งในช่วงเวลาสั้นๆ เช่น 2-5 ปีข้างหน้า เนื่องจากมีหลายปัจจัยที่ส่งผลต่อการเกิดสงครามระหว่างประเทศสองแห่งนี้ เช่น ปัจจัยเศรษฐกิจ การเมือง ปัจจัยทางสังคม และปัจจัยทางทหาร เป็นต้น ดังนั้น การทำนายเกี่ยวกับการเกิดสงครามระหว่างสหรัฐกับจีนยังมีความไม่แน่นอนอยู่เสมอ\n\nอย่างไรก็ตาม สถานการณ์ระหว่างสหรัฐกับจีนในปัจจุบันยังคงมีความเสี่ยงที่จะเกิดขึ้นและทำให้สองประเทศมาอยู่ในสถานการณ์ที่ด้อยกว่าสถานการณ์ปัจจุบัน แต่การทำนายเกี่ยวกับการเกิดสงครามก็ไม่ควรเป็นเป้าหมายหลัก เนื่องจากการเตรียมความพร้อมและการสร้างความเข้าใจและการทำงานร่วมกันระหว่างสหรัฐกับจีนยังคงเป็นทางเลือกที่ดีที่สุดในการป้องกันการเกิดสงครามระหว่างสองประเทศในอนาคต
45,th,37b86e72-d229-4c48-a2a8-37f8716e3177,37b86e72-d229-4c48-a2a8-37f8716e3177,37b86e72-d229-4c48-a2a8-37f8716e3177,2,4,เผลอนอกใจเเฟน เเต่อยากได้เขากลับมาทำไงดี,แสดงเจตนารมณ์ของคุณด้วยการขอโทษอย่างจริงใจ และอดทนรอแฟนให้อภัย แต่ต้องสำนึกไว้เสมอว่า แฟนอาจจะไม่กลับมาแล้ว หรือความรักของคุณจะไม่เหมือนเดิมอีกต่อไป
46,th,fc7d2e64-b334-496f-a74c-19b2ee786652,fc7d2e64-b334-496f-a74c-19b2ee786652,fc7d2e64-b334-496f-a74c-19b2ee786652,3,1,อธิบายหลักการทำงานของเครื่องฟอกอากาศ ขอแบบที่เด็กห้าขวบเข้าใจ,เครื่องฟอกอากาศจะดูดอากาศที่มีฝุ่นปะปนอยู่ มาผ่านตาข่ายขนาดเล็กมากๆ เล็กพอที่จะดักเม็ดฝุ่นไว้ \nแล้วอากาศสะอาดก็จะถูกปล่อยออกไป โดยไม่มีฝุ่น\nแต่ถ้าฝุ่นเล็กกว่ารูตาข่าย ก็จะยังหลุดรอดออกไปได้
47,th,08cf8e85-8ad6-44a0-a418-b28fea3f8cfb,08cf8e85-8ad6-44a0-a418-b28fea3f8cfb,08cf8e85-8ad6-44a0-a418-b28fea3f8cfb,0,0,เราสามารถใส่อิเล็กตรอนขั่วบวกหรือลบ เข้าไปในวัตถุหรือร่างกายคนได้หรือเปล่าครับ,ไม่ได้ เพราะอิเล็กตรอนถือเป็นขั้วลบ ไม่ทราบว่าคุณหมายความว่าอะไร ฉันขอขยายความเพิ่มเติม
48,th,2f9530bd-7867-455c-8a1b-7644b9d0c61c,2f9530bd-7867-455c-8a1b-7644b9d0c61c,2f9530bd-7867-455c-8a1b-7644b9d0c61c,1,2,ความตายน่ากลัวหรือไม่,ไม่น่ากลัวเลย ความตายย่อมเป็นเรื่องของธรรมชาติ
49,th,7f387170-6706-4e5b-bfd3-aa1f365ab28f,7f387170-6706-4e5b-bfd3-aa1f365ab28f,7f387170-6706-4e5b-bfd3-aa1f365ab28f,0,2,ขอคำแนะนำคะ ซื้อของขวัญจับฉลากอะไรดี งบ 400-500 บาทค่ะ,ในช่วงราคานี้มีหลายอย่างที่เป็นที่นิยม เช่น ตุ๊กตา กิ๊ฟการ์ด ของกุ๊กกิ๊ก ฯลฯ
50,th,763ef1eb-e3a7-40bc-bf20-df0737510dbb,763ef1eb-e3a7-40bc-bf20-df0737510dbb,763ef1eb-e3a7-40bc-bf20-df0737510dbb,5,2,ปิดการใช้งาน accessibility service ใน android ยังไงครับ,บริการการช่วยเหลือการเข้าถึงอาจเป็นตัวเลือกที่ดีหากคุณเป็นผู้พิการและต้องการใช้อุปกรณ์ของคุณง่ายขึ้น แต่ในขณะเดียวกันก็สร้างความกังวลว่าจะเปิดช่องให้มิจฉาชีพเข้าถึงอุปกรณ์ของคุณในทางที่ไม่ดีได้\n\nคุณสามารถทำตามขั้นตอนดังนี้เพื่อปิดบริการการช่วยเหลือการเข้าถึงในระบบ Android:\n\n1. เข้าไปที่ การตั้งค่า ในอุปกรณ์ของคุณ\n2. ขึ้นอยู่กับอุปกรณ์ของคุณ เข้าไปที่เมนูใดเมนูหนึ่งตามนี้:\n 1. การตั้งค่าเพิ่มเติม จากนั้นไปที่ การช่วยเหลือพิเศษ หรือ\n 2. การช่วยเหลือก

In [12]:
selected_df.to_csv('../data/oasst1_selected.csv', index=False)

## Extract Score from ChatGPT Response

In [146]:
import re

def remove_non_numeric_prefix(string):
  # Find the index of the first number in the string.
  index = next((i for i, c in enumerate(string) if c.isdigit()), len(string))
  # Return the substring of the string starting at the index of the first number.
  return string[index:]

def remove_non_numeric_and_space(string):
  # Create a regular expression to match all non-numeric, non-space characters.
  regex = re.compile(r"[^0-9. ]")
  # Replace all non-numeric, non-space characters with an empty string.
  return re.sub(regex, "", string)

def extract_score(text):
    l = remove_non_numeric_and_space(
        remove_non_numeric_prefix(text).split('\n')[0]
    ).split()
    if len(l)<2:
        print('Malformed scores')
        return [-1,-1]
    else:
        return [float(i) for i in l[:2]]
    
#calculate win-loss matchup
def is_second_col_better(row, first_col='human', second_col='wangchang_sft_en'):
    if row[first_col] > row[second_col]:
        return -1
    elif row[first_col] < row[second_col]:
        return 1
    else:
        return 0
    
def get_eval_stats(eval_fname):
    first_col = eval_fname.split('eval_')[-1].split('_vs_')[0]
    second_col = eval_fname.split('eval_')[-1].split('_vs_')[-1].split('.csv')[0]
    
    df = pd.read_csv(eval_fname)
    df[first_col] = df.eval_score.map(lambda x: \
        extract_score(str(x))[0]
    )
    df[second_col] = df.eval_score.map(lambda x: \
        extract_score(str(x))[1]
    )
    #filter out invalid evaluation due to API
    df = df[df[first_col]>=0].reset_index(drop=True)
    df['is_second_col_better'] = df.apply(
        lambda row: is_second_col_better(row, first_col, second_col) ,axis=1)
    
    score = df.groupby('lang')[[first_col,second_col]].agg([len,np.mean])
    score['second_over_first'] = score[second_col]['mean'] / score[first_col]['mean']
    win_loss = df.groupby('lang').is_second_col_better.value_counts(normalize=True)
    
    return score, win_loss, df

In [182]:
import ipywidgets as widgets
import glob
fnames = glob.glob('../data/eval_*.csv')

dropdown = widgets.Dropdown(options=fnames)
display(dropdown)

Dropdown(options=('../data/eval_wangchang_sft_en_vs_openthaigpt.csv', '../data/eval_wangchang_sft_enth_vs_open…

In [209]:
# Get the selected item
eval_fname = dropdown.value
score, win_loss, df = get_eval_stats(eval_fname)
score

Malformed scores
Malformed scores
Malformed scores
Malformed scores
Malformed scores
Malformed scores


wangchang_sft_en           openthaigpt           second_over_first
                  len      mean         len      mean                  
lang                                                                   
th    163              0.719632  163         0.678528  0.942882

In [210]:
win_loss

lang  is_second_col_better
th    -1                      0.552147
       1                      0.447853
Name: is_second_col_better, dtype: float64

## Human Questionnaire

We paid 200 THB per person to compare `wangchang_sft_en` vs `openthaigpt010` results for 166 Thai prompts. The annotators are randomly sourced from [this post](https://www.facebook.com/thanachart.ritbumroong/posts/pfbid02Lc1yESDLKiDpCzHNsBH5N7y7PFykE5QpUBZ2i3DG3BmospiTwyRYfgKrGVd52Vrul). The instruction to annotators are as followed:

```
ในฐานะผู้ใช้ภาษาไทย เราอยากให้คุณประเมินคำตอบของผู้ช่วยปัญญาประดิษฐ์ 2 อันที่ตอบคำสั่ง 166 ตัวอย่างว่า A ทำได้ดีกว่า, B ทำได้ดีกว่า, หรือทั้งสองอันทำได้ดี/แย่พอๆกัน เกณฑ์ในการประเมินคือ 1) ความมีประโยชน์ 2) ความเกี่ยวข้องกับคำสั่ง 3) ความถูกต้อง และ 4) รายละเอียดของคำตอบ
```

The annotators do not know which model is A nor B.

In [153]:
import glob
import pandas as pd
pd.set_option('display.max_colwidth', 0)
from tqdm.auto import tqdm
fnames = glob.glob('../data/human_questionnaire/*.csv')
fnames

dfs = []
for fname in tqdm(fnames):
    d = pd.read_csv(fname)
    d['fname'] = fname.split('/')[-1][:-4]
    dfs.append(d)
df = pd.concat(dfs).reset_index(drop=True)

  0%|          | 0/18 [00:00<?, ?it/s]

In [154]:
df.groupby('fname')['คำตอบไหนดีกว่ากัน'].count()

fname
chp    166
clt    166
ctj    166
jjj    166
jrn    166
jtm    166
krk    166
ktb    166
npp    166
prn    166
rbb    166
sck    166
srs    166
stk    166
thk    166
tsp    166
vna    166
vnt    166
Name: คำตอบไหนดีกว่ากัน, dtype: int64

In [156]:
df['คำตอบไหนดีกว่ากัน'].value_counts(normalize=True)

A ทำได้ดีกว่า    0.569277
ดี/แย่พอกัน      0.328983
B ทำได้ดีกว่า    0.101740
Name: คำตอบไหนดีกว่ากัน, dtype: float64

In [157]:
df.groupby(['fname','คำตอบไหนดีกว่ากัน']).message_id.count().reset_index()\
    .pivot_table(index=['fname',], columns='คำตอบไหนดีกว่ากัน')/166

message_id                          
คำตอบไหนดีกว่ากัน A ทำได้ดีกว่า B ทำได้ดีกว่า ดี/แย่พอกัน
fname                                                    
chp                0.403614      0.120482      0.475904  
clt                0.548193      0.036145      0.415663  
ctj                0.548193      0.096386      0.355422  
jjj                0.650602      0.036145      0.313253  
jrn                0.469880      0.048193      0.481928  
jtm                0.710843      0.054217      0.234940  
krk                0.777108      0.114458      0.108434  
ktb                0.379518      0.018072      0.602410  
npp                0.662651      0.210843      0.126506  
prn                0.686747      0.126506      0.186747  
rbb                0.403614      0.060241      0.536145  
sck                0.620482      0.084337      0.295181  
srs                0.626506      0.174699      0.198795  
stk                0.415663      0.120482      0.463855  
thk                0.602410      0.126506      0.271084  
tsp                0.632530      0.337349      0.030120  
vna                0.566265      0.030120      0.403614  
vnt                0.542169      0.036145      0.421687

In [158]:
df.fname.nunique()

18